In [ ]:
from bs4 import BeautifulSoup
import requests

## Thực hành một số trang web khác
Cách phân tích data từ 1 trang web dựa vào inspect và HTML tag<br>
Tìm đối tượng muốn trích xuất thông tin<br>
Tìm `href`, `class`, ...

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/75.0.3770.80 Safari/537.36"
}
res = requests.get("https://vietstock.vn/", headers=headers)

In [ ]:
res

<Response [200]>

In [ ]:
soup = BeautifulSoup(res.content, "html.parser")

In [ ]:
news = soup.findAll("div", {"class": "single_post_text"})

In [ ]:
len(news)

94

In [ ]:
news[12].h4.a.attrs["href"]

'/2022/07/de-rong-cat-canh-761-984064.htm'

In [ ]:
import urllib.parse

In [ ]:
sub_url = urllib.parse.urljoin("https://vietstock.vn/", '/2022/07/de-rong-cat-canh-761-984064.htm')
sub_url

'https://vietstock.vn/2022/07/de-rong-cat-canh-761-984064.htm'

In [ ]:
sub_page_res = requests.get(sub_url, headers=headers)

In [ ]:
sub_page_res

<Response [200]>

In [ ]:
sub_soup = BeautifulSoup(sub_page_res.content, "html.parser")

In [ ]:
sub_soup.find("p", {"class": "pHead"}).text

'Trong tuần qua có một thông tin đáng chú ý: Trong khi Bộ Nông nghiệp và Phát triển nông thôn vẫn chưa triển khai Đề án sản xuất 1 triệu héc ta lúa chất lượng cao ở đồng bằng sông Cửu Long (ĐBSCL) thì tổng diện tích sản xuất giống lúa chất lượng cao ở đây trong vụ hè thu 2022 đã vượt con số 1 triệu héc ta.'

In [ ]:
tags = sub_soup.find("div", {"class": "tags mt20"})

In [ ]:
tags.findAll("a")

[<a href="/tag/Vùng-đất-9-rồng">Vùng đất 9 rồng</a>,
 <a href="/tag/-Vùng-đất-rồng-cất-cánh"> Vùng đất rồng cất cánh</a>,
 <a href="/tag/-Quy-hoạch-phát-triển-ĐBSCL"> Quy hoạch phát triển ĐBSCL</a>]

## Selenium
Install `selenium` bằng `pip install selenium`<br>
Download webdriver (Chrome) nhưng phải cùng version với browser đang dùng

In [18]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
driver = webdriver.Chrome(executable_path='/Users/binhna/projects/resource/chromedriver')
username = input()
password = input()
driver.get("https://trade.vndirect.com.vn/chung-khoan/danh-muc")
user_name = driver.find_element("xpath", "\\div[@class='form-field']/input[@name='username']")
user_name.send_keys(username)
pass_word = driver.find_element("xpath", "\\div[@class='form-field']/input[@name='password']")
pass_word.send_keys(password)

/var/folders/0l/dvl_p7qs4yx7y2v7plhbhd800000gn/T/ipykernel_7882/2884912774.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/binhna/projects/resource/chromedriver')


InvalidSelectorException: Message: invalid selector: Unable to locate an element with the xpath expression \div[@class='form-field']/input[@name='username'] because of the following error:
SyntaxError: Failed to execute 'evaluate' on 'Document': The string '\div[@class='form-field']/input[@name='username']' is not a valid XPath expression.
  (Session info: chrome=104.0.5112.79)
Stacktrace:
0   chromedriver                        0x00000001006f1d60 chromedriver + 3792224
1   chromedriver                        0x0000000100688874 chromedriver + 3360884
2   chromedriver                        0x0000000100389184 chromedriver + 217476
3   chromedriver                        0x000000010038bbb4 chromedriver + 228276
4   chromedriver                        0x000000010038ba1c chromedriver + 227868
5   chromedriver                        0x000000010038bc50 chromedriver + 228432
6   chromedriver                        0x00000001003b8cbc chromedriver + 412860
7   chromedriver                      

In [17]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

driver = webdriver.Chrome(executable_path='/Users/binhna/projects/resource/chromedriver')
driver.get("http://www.python.org")
assert "Python" in driver.title

/var/folders/0l/dvl_p7qs4yx7y2v7plhbhd800000gn/T/ipykernel_7882/567735100.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path='/Users/binhna/projects/resource/chromedriver')
